In [23]:
import torch
from lab_gatr import PointCloudPoolingScales, LaBGATr
import torch_geometric as pyg
from gatr.interface import embed_oriented_plane, extract_translation
import os
import numpy as np


In [25]:
def load_point_clouds(directory='./3d_point_cloud_dataset'):
    point_clouds = []
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)
            cloud_points = []

            with open(file_path, 'r') as f:
                for line in f:
                    # Skip lines that do not contain numeric values
                    try:
                        # Try to convert the line to a float, meaning it's a numeric line
                        point = list(map(float, line.split()))
                        if len(point) == 3:  # Ensure it's a 3D point
                            cloud_points.append(point)
                    except ValueError:
                        # Skip the line if it can't be converted to floats
                        continue

            # Convert the list of points to a numpy array and append to the dataset
            point_clouds.append(np.array(cloud_points))

    return point_clouds

point_clouds = load_point_clouds()
print(f"Loaded {len(point_clouds)} point clouds.")


Loaded 1500 point clouds.


In [26]:
def prepare_data_for_model(point_clouds):
    pos = []
    orientation = []
    scalar_feature = []
    
    for cloud in point_clouds:
        # Example: Assume that we use the first point as the position and normal as the orientation
        pos.append(cloud[0])  # First point as the position (this is just an example)
        orientation.append(np.random.rand(3))  # Random orientation (replace with real logic)
        scalar_feature.append(np.random.rand(1))  # Random scalar (replace with real data)

    pos = torch.tensor(np.array(pos), dtype=torch.float32)
    orientation = torch.tensor(np.array(orientation), dtype=torch.float32)
    scalar_feature = torch.tensor(np.array(scalar_feature), dtype=torch.float32)

    return pyg.data.Data(pos=pos, orientation=orientation, scalar_feature=scalar_feature)

# Prepare the data
data = prepare_data_for_model(point_clouds)
print(f"Data prepared with {data.pos.shape[0]} samples.")


Data prepared with 1500 samples.


In [27]:
class GeometricAlgebraInterface:
    num_input_channels = num_output_channels = 1
    num_input_scalars = num_output_scalars = 1

    @staticmethod
    @torch.no_grad()
    def embed(data):
        # Embed the point cloud into multivectors and scalars (your model-specific logic here)
        multivectors = embed_oriented_plane(normal=data.orientation, position=data.pos).view(-1, 1, 16)
        scalars = data.scalar_feature.view(-1, 1)
        return multivectors, scalars

    @staticmethod
    def dislodge(multivectors, scalars):
        return extract_translation(multivectors).squeeze()


In [ ]:
model = LaBGATr(GeometricAlgebraInterface, d_model=8, num_blocks=10, num_attn_heads=4, use_class_token=False)

# Print the model architecture
print(model)


# 5. Load and preprocess point cloud data
point_clouds = load_point_clouds('./3d_point_cloud_dataset')
print(f"Loaded {len(point_clouds)} point clouds.")

# Prepare data for the model
data = prepare_data_for_model(point_clouds)
print(f"Data prepared with {data.pos.shape[0]} samples.")


# 6. Run the model with the prepared data
output = model(data)
print(f"Output shape: {output.shape}")


# 7. Surrogate regression task: Compute loss (Dummy target values for regression)
target = torch.rand_like(output)  # Replace with actual ground truth labels

# Compute the regression loss (e.g., Mean Squared Error)
loss_fn = torch.nn.MSELoss()
loss = loss_fn(output, target)
print(f"Regression Loss: {loss.item()}")

LaB-GATr (261761 parameters)
LaBGATr(
  (tokeniser): CrossAttentionTokeniser(
    (point_cloud_pooling): PointCloudPooling()
    (mlp): MLP(
      (linear_layers): ModuleList(
        (0): GeometricBilinear(
          (linear_left): EquiLinear(
            (s2mvs): Linear(in_features=2, out_features=4, bias=True)
          )
          (linear_right): EquiLinear(
            (s2mvs): Linear(in_features=2, out_features=4, bias=True)
          )
          (linear_join_left): EquiLinear(
            (s2mvs): Linear(in_features=2, out_features=4, bias=True)
          )
          (linear_join_right): EquiLinear(
            (s2mvs): Linear(in_features=2, out_features=4, bias=True)
          )
          (linear_out): EquiLinear(
            (s2mvs): Linear(in_features=2, out_features=8, bias=True)
            (mvs2s): Linear(in_features=8, out_features=32, bias=True)
            (s2s): Linear(in_features=2, out_features=32, bias=False)
          )
        )
        (1): EquiLinear(
          

AttributeError: 'GlobalStorage' object has no attribute 'scale0_sampling_index'

In [29]:
# Forward pass through the model
output = model(data)
print(f"Output shape: {output.shape}")


AttributeError: 'GlobalStorage' object has no attribute 'scale0_sampling_index'

In [30]:
# Dummy target values for regression (replace with actual ground truth)
target = torch.rand_like(output)

# Compute a regression loss (e.g., Mean Squared Error)
loss_fn = torch.nn.MSELoss()
loss = loss_fn(output, target)
print(f"Regression Loss: {loss.item()}")


NameError: name 'output' is not defined